In [ ]:
# %pip install torch

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import requests
import numpy as np

import os


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
## set environment variables
# os.environ["APIKEY"] = "*************"

## Thrifty-Rabbit Project
### Project Goals

Goals:
1. Monitor and evaluate a dividend portfolio.
2. Create an optimized value portfolio.



#### Data Import and Processing

Step one is to have a way to get stock prices and fundamental data needed to evaulate an asset into our program. 

Alpha-vantage offers a free api that will allow me to get all the data I need to analyze an asset. 

I will use three different endpoints to collect the data needed to analyze the asset:


In [2]:
# get the monthly adjusted prices for the portfolio symbols using AlphaVantage API

prices = pd.DataFrame()
for symbol in portfolio:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&apikey=demo"
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data["Monthly Adjusted Time Series"]).T
    df["symbol"] = symbol
    prices = prices.append(df)

# rename the columns and set the index as date
prices.columns = ["open", "high", "low", "close", "adj_close", "volume", "dividend", "symbol"]
prices.index = pd.to_datetime(prices.index)
prices.sort_index(inplace=True)

# print the first five rows of the dataframe
print(prices.head())

In [ ]:
print(os.getenv("APIKEY"))

## Analysis

In [ ]:
#Build portfolio
portfolio = [
    "SPY",
    "WMT",
    "JNPR",
    "CVS",
    "BAC",
    "JNJ",
    "ABB",
    "ORCL",
] 